In [1]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import *
from openmmtools.states import *
from openmmtools.integrators import *
from simtk.openmm import CompoundIntegrator
from openmmtools import alchemy
from simtk import unit
from openmmtools.utils import RestorableOpenMMObject

import numpy as np
import copy, sys, time, os, math
import nglview
import logging, math
from simtk import openmm
import parmed
from openmmtools.utils import SubhookedABCMeta, Timer, RestorableOpenMMObject
from openmmtools import testsystems, alchemy
import netCDF4 as nc
from blues.integrators import AlchemicalExternalLangevinIntegrator
import mdtraj

finfo = np.finfo(np.float32)
rtol = finfo.precision
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.getLogger("openmmtools.alchemy").setLevel(logging.ERROR)
np.random.RandomState(seed=3134)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

/home/nathanlim/anaconda3/envs/openmmtools/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nathanlim/anaconda3/envs/openmmtools/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Below try it inside classes

In [2]:
class DummySimulation(object):
    def __init__(self, integrator, system, topology):
        self.integrator = integrator
        self.topology = topology
        self.system = system

In [3]:
class NCMCMove(MetropolizedMove):
    
    def __init__(self, atom_subset=None, context_cache=None, n_steps=1000, n_restart_attempts=4):
        self.n_steps = n_steps
        self.n_accepted = 0
        self.n_proposed = 0
        self.logp_accept = 0
        self.initial_energy = None
        self.initial_positions = None
        self.final_energy = None
        self.final_positions = None
        self.proposed_positions = None
        self.n_restart_attempts = n_restart_attempts
        self.atom_subset = atom_subset
        self.context_cache = context_cache
        
    @property
    def statistics(self):
        """The acceptance statistics as a dictionary."""
        return dict(n_accepted=self.n_accepted, 
                    n_proposed=self.n_proposed,
                    initial_energy=self.initial_energy,
                    initial_positions=self.initial_positions,
                   final_energy=self.final_energy,
                   proposed_positions=self.proposed_positions,
                   final_positions=self.final_positions,
                   logp_accept=self.logp_accept)

    @statistics.setter
    def statistics(self, value):
        self.n_accepted = value['n_accepted']
        self.n_proposed = value['n_proposed']
        self.initial_energy = value['initial_energy']
        self.initial_positions = value['initial_positions']
        self.final_energy = value['final_energy']
        self.proposed_positions = value['proposed_positions']
        self.final_positions = value['final_positions']
        self.logp_accept = value['logp_accept']
        
    def _propose_positions(self, positions, center_of_mass):
        """Return new proposed positions.

        These method must be implemented in subclasses.

        Parameters
        ----------
        positions : nx3 numpy.ndarray
            The original positions of the subset of atoms that these move
            applied to.

        Returns
        -------
        proposed_positions : nx3 numpy.ndarray
            The new proposed positions.

        """
        
        print('Proposing positions...')
        
        reduced_pos = positions - center_of_mass
        # Define random rotational move on the ligand
        rand_quat = mdtraj.utils.uniform_quaternion(size=None, random_state=np.random.RandomState(seed=3134))
        rand_rotation_matrix = mdtraj.utils.rotation_matrix_from_quaternion(rand_quat)
        #multiply lig coordinates by rot matrix and add back COM translation from origin
        proposed_positions = np.dot(reduced_pos, rand_rotation_matrix) * positions.unit + center_of_mass
        
        return proposed_positions
      
    
    def _create_dummy_simulation(self, integrator, system, topology):
        """
        Generate a dummy Simulation object because the Reporter
        expects an `openmm.Simulation` object in order to report information
        from it's respective attached integrator/
        """
        return DummySimulation(integrator, system, topology) 
    
    def _before_integration(self, context, integrator, thermodynamic_state):
        """Execute code after Context creation and before integration."""
        
        return self._create_dummy_simulation(integrator,
                                            thermodynamic_state.get_system(), 
                                            thermodynamic_state.topology)
    
    def _after_integration(self, context, thermodynamic_state):
        """Implement BaseIntegratorMove._after_integration()."""
        integrator = context.getIntegrator()
        try:
            # Accumulate acceptance statistics.
            ghmc_global_variables = {integrator.getGlobalVariableName(index): index
                                 for index in range(integrator.getNumGlobalVariables())}
            n_accepted = integrator.getGlobalVariable(ghmc_global_variables['naccept'])
            n_proposed = integrator.getGlobalVariable(ghmc_global_variables['ntrials'])
            self.n_accepted += n_accepted
            self.n_proposed += n_proposed
        except Exception as e:
            print(e)
            pass
        
    def _get_integrator(self, thermodynamic_state):
        return AlchemicalExternalLangevinIntegrator(
            alchemical_functions={ 'lambda_sterics':
                'min(1, (1/0.3)*abs(lambda-0.5))',
                'lambda_electrostatics':
                'step(0.2-lambda) - 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)'},
            splitting="H V R O R V H",
            temperature=thermodynamic_state.temperature,
            nsteps_neq=self.n_steps,
            timestep=4.0 * unit.femtoseconds,
            nprop=3,
            prop_lambda=0.3)
    
    def getMasses(self, atom_subset, topology):
        """
        Returns a list of masses of the specified ligand atoms.
        Parameters
        ----------
        topology: parmed.Topology
            ParmEd topology object containing atoms of the system.
        Returns
        -------
        masses: 1xn numpy.array * simtk.unit.dalton
            array of masses of len(self.atom_indices), denoting
            the masses of the atoms in self.atom_indices
        totalmass: float * simtk.unit.dalton
            The sum of the mass found in masses
        """
        atoms = [ list(topology.atoms())[i] for i in atom_subset]
        masses = unit.Quantity(np.zeros([int(len(atoms)), 1], np.float32), unit.dalton)
        for idx, atom in enumerate(atoms):
            masses[idx] = atom.element._mass
        totalmass = masses.sum()
        return masses, totalmass

    def getCenterOfMass(self, positions, masses):
        """Returns the calculated center of mass of the ligand as a numpy.array
        Parameters
        ----------
        positions: nx3 numpy array * simtk.unit compatible with simtk.unit.nanometers
            ParmEd positions of the atoms to be moved.
        masses : numpy.array
            numpy.array of particle masses
        Returns
        -------
        center_of_mass: numpy array * simtk.unit compatible with simtk.unit.nanometers
            1x3 numpy.array of the center of mass of the given positions
        """
        coordinates = np.asarray(positions._value, np.float32)
        center_of_mass = parmed.geometry.center_of_mass(coordinates, masses) * positions.unit
        return center_of_mass
    
    def apply(self, thermodynamic_state, sampler_state, reporter):
        """Apply a metropolized move to the sampler state.

        Total number of acceptances and proposed move are updated.

        Parameters
        ----------
        thermodynamic_state : openmmtools.states.ThermodynamicState
           The thermodynamic state to use to apply the move.
        sampler_state : openmmtools.states.SamplerState
           The initial sampler state to apply the move to. This is modified.

        """
        timer = Timer()
        benchmark_id = 'Applying {}'.format(self.__class__.__name__ )
        timer.start(benchmark_id)

        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            context_cache = self.context_cache
            
        # Create integrator
        integrator = self._get_integrator(thermodynamic_state)

        # Create context
        context, integrator = context_cache.get_context(thermodynamic_state, integrator)
        simulation = self._create_dummy_simulation(integrator,
                                            thermodynamic_state.get_system(), 
                                            thermodynamic_state.topology)
        self.kT = integrator.kT
        
        # Compute initial energy. We don't need to set velocities to compute the potential.
        # TODO assume sampler_state.potential_energy is the correct potential if not None?
        sampler_state.apply_to_context(context, ignore_velocities=True)
        initial_energy = thermodynamic_state.reduced_potential(context)
        print('NCMC_PE_i:', initial_energy)
        
        masses, totalmass = self.getMasses(self.atom_subset, thermodynamic_state.topology)
        # Handle default and weird cases for atom_subset.
        if self.atom_subset is None:
            atom_subset = slice(None)
        elif not isinstance(self.atom_subset, slice) and len(self.atom_subset) == 1:
            # Slice so that initial_positions (below) will have a 2D shape.
            atom_subset = slice(self.atom_subset[0], self.atom_subset[0]+1)
        else:
            atom_subset = slice(self.atom_subset[0], self.atom_subset[-1]+1)#self.atom_subset
        
        # Store initial positions of the atoms that are moved.
        # We'll use this also to recover in case the move is rejected.
        if isinstance(atom_subset, slice):
            # Numpy array when sliced return a view, they are not copied.
            initial_positions = copy.deepcopy(sampler_state.positions[atom_subset])
        else:
            # This automatically creates a copy.
            initial_positions = sampler_state.positions[atom_subset]
            
        print("NCMC_i", initial_positions)
        
        # Calculate the center of mass
        center_of_mass = self.getCenterOfMass(initial_positions, masses)
        # Perform the integration.
        for attempt_counter in range(self.n_restart_attempts + 1):
        
            try:
                # Run dynamics
                #NML: Do in 1 steps for debugging
                timer.start("{}: step({})".format(benchmark_id, n_steps))
                rotation_step = int(self.n_steps/2)
                for n in range(self.n_steps+1):
                    integrator.step(1)

                    step = integrator.getGlobalVariableByName('step')
                    alch_lambda = integrator.getGlobalVariableByName('lambda')
                    lambda_step = integrator.getGlobalVariableByName('lambda_step')
                    protocol_work = integrator.getGlobalVariableByName('protocol_work')

                    thermodynamic_state.set_alchemical_variable('lambda', alch_lambda)
                    thermodynamic_state.apply_to_context(context)

                    #lambda_sterics = context.getParameter('lambda_sterics')
                    #lambda_electrostatics = context.getParameter('lambda_electrostatics')

                    if n == rotation_step:
                        # Propose perturbed positions. Modifying the reference changes the sampler state.
                        proposed_positions = self._propose_positions(initial_positions, center_of_mass)
                        # Compute the energy of the proposed positions.
                        sampler_state.positions[atom_subset] = proposed_positions
                        sampler_state.apply_to_context(context, ignore_velocities=True)

                    # Get the context state for reporting
                    context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                             enforcePeriodicBox=thermodynamic_state.is_periodic)
                    #print('IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step) 
                    #print('\t'+'ContextParameters:', lambda_step, 'Sterics:', lambda_sterics, 'Elec:', lambda_electrostatics)     
                    if n % 25 == 0:
                        print('NCMC---IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step)
                        #print('Reporting NCMC step:', n)
                        #reporter.report(simulation, context_state)

                    if step == self.n_steps:
                        break
            except Exception as e:
                print(e)
                # Catches particle positions becoming nan during integration.
                restart = True
            else:
                timer.stop("{}: step({})".format(benchmark_id, self.n_steps))   


                # Subclasses can read here info from the context to update internal statistics.
                #self._after_integration(context, thermodynamic_state)

                # Updated sampler state.
                #timer.start("{}: update sampler state".format(benchmark_id))    
                # This is an optimization around the fact that Collective Variables are not a part of the State,
                # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
                # the State.

                 # Update everything but the collective variables from the State object
                sampler_state.update_from_context(context_state, ignore_collective_variables=True, ignore_velocities=True)
                # Update only the collective variables from the Context
                sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                                  ignore_collective_variables=False)                                          
                #timer.stop("{}: update sampler state".format(benchmark_id))
                context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                     enforcePeriodicBox=thermodynamic_state.is_periodic)
                # Check for NaNs in energies.
                potential_energy = context_state.getPotentialEnergy()
                restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

            # Restart the move if we found NaNs.
            if restart:
                err_msg = ('Potential energy is NaN after {} attempts of integration '
                           'with move {}'.format(attempt_counter, self.__class__.__name__))

                # If we are on our last chance before crash, try to re-initialize context
                if attempt_counter == self.n_restart_attempts - 1:
                    logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                    context.reinitialize()
                    sampler_state.apply_to_context(context)
                    thermodynamic_state.apply_to_context(context)
                # If we have hit the number of restart attempts, raise an exception.
                elif attempt_counter == self.n_restart_attempts:
                    # Restore the context to the state right before the integration.
                    sampler_state.apply_to_context(context)
                    logger.error(err_msg)
                    raise IntegratorMoveError(err_msg, self, context)
                else:
                    logger.warning(err_msg + ' Attempting a restart...')
            else:
                break
            
        final_energy = thermodynamic_state.reduced_potential(context)
        print('NCMC_PE_f', final_energy)
        final_positions = sampler_state.positions[atom_subset]
        logp_accept = context._integrator.getLogAcceptanceProbability(context)
        print("Log Acceptance P:", logp_accept)
        
        self.initial_positions = initial_positions
        self.initial_energy = initial_energy
        self.proposed_positions = proposed_positions
        self.final_energy = final_energy
        self.final_positions = final_positions
        self.logp_accept = logp_accept
        

        # Print timing information.
        timer.stop(benchmark_id)
        #timer.report_timing()


In [4]:
class ModLangevinDynamicsMove(LangevinDynamicsMove):
    
    def _create_dummy_simulation(self, integrator, system, topology):
        """
        Generate a dummy Simulation object because the Reporter
        expects an `openmm.Simulation` object in order to report information
        from it's respective attached integrator/
        """
        return DummySimulation(integrator, system, topology) 
    

    def _before_integration(self, context, thermodynamic_state):
        """Execute code after Context creation and before integration."""
        self.initial_context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                             enforcePeriodicBox=thermodynamic_state.is_periodic)
        self.initial_energy = thermodynamic_state.reduced_potential(context)
        print("MD_PE_i", self.initial_energy)
        
    def _after_integration(self, context, thermodynamic_state):
        """Execute code after integration.

        After this point there are no guarantees that the Context will still
        exist, together with its bound integrator and system.
        """
        self.final_context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                     enforcePeriodicBox=thermodynamic_state.is_periodic)
        self.final_energy = thermodynamic_state.reduced_potential(context)
        print("MD_PE_f", self.final_energy)
        
    def apply(self, thermodynamic_state, sampler_state, reporter):
        """Propagate the state through the integrator.

        This updates the SamplerState after the integration. It also logs
        benchmarking information through the utils.Timer class.

        Parameters
        ----------
        thermodynamic_state : openmmtools.states.ThermodynamicState
           The thermodynamic state to use to propagate dynamics.
        sampler_state : openmmtools.states.SamplerState
           The sampler state to apply the move to. This is modified.

        See Also
        --------
        openmmtools.utils.Timer

        """
        move_name = self.__class__.__name__  # shortcut
        timer = Timer()

        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            context_cache = self.context_cache

        # Create integrator.
        integrator = self._get_integrator(thermodynamic_state)

        # Create context.
        timer.start("{}: Context request".format(move_name))
        context, integrator = context_cache.get_context(thermodynamic_state, integrator)
        timer.stop("{}: Context request".format(move_name))
        #logger.debug("{}: Context obtained, platform is {}".format(
        #    move_name, context.getPlatform().getName()))
        simulation = self._create_dummy_simulation(integrator,
                                            thermodynamic_state.get_system(), 
                                            thermodynamic_state.topology)
        # Perform the integration.
        for attempt_counter in range(self.n_restart_attempts + 1):

            # If we reassign velocities, we can ignore the ones in sampler_state.
            sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
            if self.reassign_velocities:
                print("Resetting velocities in MD")
                context.setVelocitiesToTemperature(thermodynamic_state.temperature)

            # Subclasses may implement _before_integration().
            self._before_integration(context, thermodynamic_state)

            try:
                # Run dynamics.
                timer.start("{}: step({})".format(move_name, self.n_steps))
                for n in range(self.n_steps):
                    integrator.step(1)
                    
                    if n % 25 == 0: 
                        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                     enforcePeriodicBox=thermodynamic_state.is_periodic)
                        print('Reporting MD step:', n)
                        reporter.report(simulation, context_state)
                        
            except Exception as e:
                print(e)
                # Catches particle positions becoming nan during integration.
                restart = True
            else:
                timer.stop("{}: step({})".format(move_name, self.n_steps))

                # We get also velocities here even if we don't need them because we
                # will recycle this State to update the sampler state object. This
                # way we won't need a second call to Context.getState().
                context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)

                # Check for NaNs in energies.
                potential_energy = context_state.getPotentialEnergy()
                restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

            # Restart the move if we found NaNs.
            if restart:
                err_msg = ('Potential energy is NaN after {} attempts of integration '
                           'with move {}'.format(attempt_counter, self.__class__.__name__))

                # If we are on our last chance before crash, try to re-initialize context
                if attempt_counter == self.n_restart_attempts - 1:
                    logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                    context.reinitialize()
                    sampler_state.apply_to_context(context)
                    thermodynamic_state.apply_to_context(context)
                # If we have hit the number of restart attempts, raise an exception.
                elif attempt_counter == self.n_restart_attempts:
                    # Restore the context to the state right before the integration.
                    sampler_state.apply_to_context(context)
                    logger.error(err_msg)
                    raise IntegratorMoveError(err_msg, self, context)
                else:
                    logger.warning(err_msg + ' Attempting a restart...')
            else:
                break

        # Subclasses can read here info from the context to update internal statistics.
        self._after_integration(context, thermodynamic_state)

        # Updated sampler state.
        timer.start("{}: update sampler state".format(move_name))
        # This is an optimization around the fact that Collective Variables are not a part of the State,
        # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
        # the State.
        # Update everything but the collective variables from the State object
        sampler_state.update_from_context(context_state, ignore_collective_variables=True)
        # Update only the collective variables from the Context
        sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                          ignore_collective_variables=False)
        timer.stop("{}: update sampler state".format(move_name))

        #timer.report_timing()

In [5]:
class NCMCSampler(object):


    def __init__(self, atom_subset=None, thermodynamic_state=None, alch_thermodynamic_state=None, 
                 sampler_state=None, move=None, platform=None, reporter=None, 
                 pdbfile=None, topology=None):
        """
        Create an MCMC sampler.
        Parameters
        ----------
        thermodynamic_state : ThermodynamicState
            The thermodynamic state to simulate
        sampler_state : SamplerState
            The initial sampler state to simulate from.
        platform : simtk.openmm.Platform, optional, default=None
            If specified, this platform will be used
        ncfile : netCDF4.Dataset, optional, default=None
            NetCDF storage file.
        """

        if alch_thermodynamic_state is None:
            raise Exception("'alch_thermodynamic_state' must be specified")
        if thermodynamic_state is None:
            raise Exception("'thermodynamic_state' must be specified")
        if sampler_state is None:
            raise Exception("'sampler_state' must be specified")
        
        self.atom_subset = atom_subset
        # Make a deep copy of the state so that initial state is unchanged.    
        self.alch_thermodynamic_state = copy.deepcopy(alch_thermodynamic_state)
        self.thermodynamic_state = copy.deepcopy(thermodynamic_state)
        self.sampler_state = copy.deepcopy(sampler_state)
        self.ncmc_move = move[1]
        self.langevin_move = move[0]
        
        #NML: Attach topology to thermodynamic_states
        self.alch_thermodynamic_state.topology = topology
        self.thermodynamic_state.topology = topology    
        
        # Initialize
        self.n_accepted = 0 
        #self.reject = 0
        self.n_proposed = 0
        
        self.verbose = True
        self.platform = platform

        # For writing trajectory files
        self.reporter = reporter
        self._timing = dict()
        
    def _stepNCMC(self):
        # Run NCMC
        self.ncmc_move.apply(self.alch_thermodynamic_state, self.sampler_state, self.reporter)
        # Initial positions in NCMC should match final of MD
        #print("NCMC_i", self.ncmc_move.initial_positions)
        # Final positions of NCMC (should be different than proposed)
        print("NCMC_f", self.sampler_state.positions[self.atom_subset])
         
    def _acceptRejectMove(self):
        # Get alch_PE by using the thermodynamic state from MD 
        # with final positions from NCMC simulations
        alch_energy = self.thermodynamic_state.reduced_potential(self.sampler_state)
        print("Alch energy", alch_energy)
        correction_factor = (self.ncmc_move.initial_energy - self.langevin_move.initial_energy 
                     + alch_energy - self.ncmc_move.final_energy) #* (-1.0 / self.ncmc_move.kT) * unit.kilojoules_per_mole
        # Accept or reject with Metropolis criteria.
        #delta_energy = self.ncmc_move.final_energy - self.ncmc_move.initial_energy
        #print("dE {} + corr {}".format(delta_energy, correction_factor))
        #delta_energy = delta_energy + correction_factor
        #print("dE", delta_energy)
        #work_ncmc = np.exp(-delta_energy)
        #print("work_ncmc:", work_ncmc)
        #randnum = np.random.rand()
        
        work_ncmc = self.ncmc_move.logp_accept
        randnum = math.log(np.random.random())
        print("work_ncmc {} + corr {}".format(work_ncmc, correction_factor))
        work_ncmc = work_ncmc + correction_factor
        print("work_ncmc", work_ncmc)
        if (not np.isnan(work_ncmc) and work_ncmc > randnum):
            print('NCMC MOVE ACCEPTED: work_ncmc {} > randnum {}'.format(work_ncmc, randnum))
            self.n_accepted += 1
        else:
            print('NCMC MOVE REJECTED: work_ncmc {} < {}'.format(work_ncmc, randnum))
            #print('Sampler_state positions:',self.sampler_state.positions[self.atom_subset])
            # Restore original positions.
            self.sampler_state.positions[self.atom_subset] = self.ncmc_move.initial_positions
            #print('Restored Sampler_state positions:',self.sampler_state.positions[self.atom_subset])
            
            
            # Potential energy should be from last MD step in the previous iteration
            #print(self.sampler_state.positions)
            #potential_energy = self.thermodynamic_state.reduced_potential(self.langevin_move.final_context_state)
            #print(potential_energy)
            #sys.exit(1)
           
            #if not math.isclose(self.langevin_move.final_energy, potential_energy, rel_tol=float('1e-%s' % rtol)):
             #   print(
             #       'Last MD potential energy %s != Current MD potential energy %s. Potential energy should match the prior state.'
             #       % (self.langevin_move.final_energy, potential_energy))
             #   sys.exit(1)
            
    def _stepMD(self):
        # Run langevin dynamics
        self.langevin_move.initial_positions = self.sampler_state.positions[self.atom_subset]
        # Potential energy should be from last MD step in the previous iteration
        print("MD_i", self.langevin_move.initial_positions)
        self.langevin_move.apply(self.thermodynamic_state, self.sampler_state, self.reporter)
        self.langevin_move.final_positions = self.sampler_state.positions[self.atom_subset]
        print("MD_f", self.langevin_move.final_positions)   
        
    def update(self):
        """
        Update the sampler with one step of sampling.
        """
        if self.verbose:
            print("." * 80)
            print("MCMC sampler iteration %d" % self.n_proposed)
            
        initial_time = time.time()
        
        self._stepNCMC()
        self._acceptRejectMove()
        self._stepMD()
    
        final_time = time.time()
        elapsed_time = final_time - initial_time
        self._timing['sample positions'] = elapsed_time 
        
        # Increment iteration count
        self.n_proposed += 1

        if self.verbose:
            print("." * 80)

    def run(self, n_iterations=1):
        """
        Run the sampler for the specified number of iterations
        Parameters
        ----------
        niterations : int, optional, default=1
            Number of iterations to run the sampler for.
        """
        self.minimize()
        self._stepMD()
        for iteration in range(n_iterations):
            self.update()
                   
    def minimize(self,
                 tolerance=1.0 * unit.kilocalories_per_mole / unit.angstroms,
                 max_iterations=100,
                 context_cache=None):
        """Minimize the current configuration.

        Parameters
        ----------
        tolerance : simtk.unit.Quantity, optional
            Tolerance to use for minimization termination criterion (units of
            energy/(mole*distance), default is 1*kilocalories_per_mole/angstroms).
        max_iterations : int, optional
            Maximum number of iterations to use for minimization. If 0, the minimization
            will continue until convergence (default is 100).
        context_cache : openmmtools.cache.ContextCache, optional
            The ContextCache to use for Context creation. If None, the global cache
            openmmtools.cache.global_context_cache is used (default is None).

        """
        if context_cache is None:
            context_cache = cache.global_context_cache
        timer = Timer()

        # Use LocalEnergyMinimizer
        timer.start("Context request")
        integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
        context, integrator = context_cache.get_context(self.thermodynamic_state, integrator)
        self.sampler_state.apply_to_context(context)
        logger.debug("LocalEnergyMinimizer: platform is %s" % context.getPlatform().getName())
        logger.debug("Minimizing with tolerance %s and %d max. iterations." % (tolerance, max_iterations))
        timer.stop("Context request")

        timer.start("LocalEnergyMinimizer minimize")
        openmm.LocalEnergyMinimizer.minimize(context, tolerance, max_iterations)
        timer.stop("LocalEnergyMinimizer minimize")

        # Retrieve data.
        self.sampler_state.update_from_context(context)
        
        context_state = context.getState(
            getPositions=True,
            getVelocities=True,
            getEnergy=True,
            enforcePeriodicBox=self.thermodynamic_state.is_periodic)
        potential_energy = context_state.getPotentialEnergy()
        print(potential_energy)


        #timer.report_timing()

In [6]:
from blues import utils   
# Define parameters
temperature = 300 * unit.kelvin
collision_rate = 1 / unit.picoseconds
timestep = 4.0 * unit.femtoseconds
n_steps = 100
reportInterval = 1000
nIter=100
prmtop = utils.get_data_filename('blues', 'tests/data/TOL-parm.prmtop')
inpcrd = utils.get_data_filename('blues', 'tests/data/TOL-parm.inpcrd')
tol = parmed.load_file(prmtop, xyz=inpcrd)
tol.system = tol.createSystem(nonbondedMethod=openmm.app.PME,
                             nonbondedCutoff=10*unit.angstrom,
                             constraints=openmm.app.HBonds,
                             hydrogenMass=3.024*unit.dalton,
                             rigidWater=True,removeCMMotion=True,
                             flexibleConstraints=True, splitDihedrals=False
                             )

2019-02-06 15:46:45,507 | INFO : Adding bonds...
2019-02-06 15:46:45,511 | INFO : Adding angles...
2019-02-06 15:46:45,513 | INFO : Adding dihedrals...
2019-02-06 15:46:45,514 | INFO : Adding Ryckaert-Bellemans torsions...
2019-02-06 15:46:45,515 | INFO : Adding Urey-Bradleys...
2019-02-06 15:46:45,515 | INFO : Adding improper torsions...
2019-02-06 15:46:45,516 | INFO : Adding CMAP torsions...
2019-02-06 15:46:45,516 | INFO : Adding trigonal angle terms...
2019-02-06 15:46:45,517 | INFO : Adding out-of-plane bends...
2019-02-06 15:46:45,517 | INFO : Adding pi-torsions...
2019-02-06 15:46:45,518 | INFO : Adding stretch-bends...
2019-02-06 15:46:45,518 | INFO : Adding torsion-torsions...
2019-02-06 15:46:45,518 | INFO : Adding Nonbonded force...


In [7]:
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False,
                                            disable_alchemical_dispersion_correction=True,
                                           alchemical_pme_treatment='direct-space')
alchemical_atom_idx = utils.atomIndexfromTop('LIG',tol.topology)
print(alchemical_atom_idx)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=alchemical_atom_idx,
                                             annihilate_sterics=False, 
                                             annihilate_electrostatics=True)
alchemical_atoms = list(alchemical_region.alchemical_atoms)
toluene_alchemical_system = factory.create_alchemical_system(
    reference_system=tol.system, alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(toluene_alchemical_system)
# Create our custom State objects
# Need two different Thermodynamic State objects
# Context cache will grab correct thermodynamic state
# Keeping them in sync is in SamplerState.apply to context
# Have apply return accumulated work

alch_thermodynamic_state = ThermodynamicState(system=toluene_alchemical_system, temperature=temperature)
alch_thermodynamic_state = CompoundThermodynamicState(alch_thermodynamic_state, composable_states=[alchemical_state])
alch_thermodynamic_state.alchemical_atoms = alchemical_atoms
alch_thermodynamic_state.topology = tol.topology
thermodynamic_state = ThermodynamicState(system=tol.system, temperature=temperature)
sampler_state = SamplerState(positions=tol.positions)
context_cache = cache.ContextCache(capacity=1)
atom_subset = slice(alchemical_atoms[0], alchemical_atoms[-1]+1)
print(atom_subset)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
slice(0, 15, None)


In [8]:
ncmc_move = NCMCMove(atom_subset=alchemical_atoms, context_cache=context_cache, n_steps=n_steps)
langevin_move = ModLangevinDynamicsMove(timestep, collision_rate, n_steps, 
                                     reassign_velocities=True, 
                                    context_cache=context_cache)

In [9]:
from blues.reporters import NetCDF4Reporter
with open('test.pdb', 'w') as pdb:
    openmm.app.pdbfile.PDBFile.writeFile(tol.topology, tol.positions, pdb)
filename = 'test.nc'
if os.path.exists(filename):
    os.remove(filename)
else:
    print("Sorry, I can not remove %s file." % filename)
nc_reporter = NetCDF4Reporter(filename, 1)
nc_reporter.reportInterval = reportInterval

In [10]:
sampler = NCMCSampler(atom_subset, thermodynamic_state, alch_thermodynamic_state, 
                 sampler_state, move=[langevin_move, ncmc_move], platform=None, reporter=nc_reporter, 
                 pdbfile=None, topology=tol.topology)

In [11]:
sampler.run(5)

2019-02-06 15:46:46,572 | DEBUG : LocalEnergyMinimizer: platform is CUDA
2019-02-06 15:46:46,573 | DEBUG : Minimizing with tolerance 1.0 kcal/(A mol) and 100 max. iterations.
-12395.331872255978 kJ/mol
MD_i [[ 1.79168597e-01  2.07402370e+00 -1.63939565e-01]
 [ 1.81629539e-01  2.07059168e+00 -1.23630073e-02]
 [ 2.87136644e-01  2.00687029e+00  5.62626831e-02]
 [ 2.90175825e-01  2.00862461e+00  1.95847034e-01]
 [ 1.88874334e-01  2.07115402e+00  2.67375052e-01]
 [ 8.46767724e-02  2.13513512e+00  2.01779917e-01]
 [ 8.00287500e-02  2.13601157e+00  6.17864281e-02]
 [ 2.19528466e-01  2.16776039e+00 -2.04139471e-01]
 [ 2.41018429e-01  1.99305445e+00 -2.04562932e-01]
 [ 7.62122422e-02  2.06331883e+00 -2.00234070e-01]
 [ 3.66474569e-01  1.95570807e+00  2.57991580e-03]
 [ 3.67811561e-01  1.95485740e+00  2.49463215e-01]
 [ 1.88850865e-01  2.06835761e+00  3.75939041e-01]
 [ 5.04606450e-03  2.18262951e+00  2.58323640e-01]
 [-1.95292977e-03  2.18827169e+00  1.34042976e-02]] nm
Resetting velocities in 

/home/nathanlim/SCRATCH/molssi-project/openmmtools/openmmtools/alchemy.py:344: UserWarning: AlchemicalState.get_alchemical_variable is deprecated. Use AlchemicalState.get_function_variable instead.
  warnings.warn('AlchemicalState.get_alchemical_variable is deprecated. '


NCMC---IntgrParameters: Step: 76.0 PE: -10792.177875903362 kJ/mol Work: 282.59375 Lambda:  0.76 Lambda Step:  152.0
NCMC_PE_f -4394.6582204890565
Log Acceptance P: -105.41045083742316
NCMC_f [[ 2.54046351e-01  1.99805558e+00  3.88712347e-01]
 [ 1.85028210e-01  2.06284285e+00  2.71906972e-01]
 [ 7.45495111e-02  1.99797010e+00  2.05030009e-01]
 [ 7.88025279e-03  2.05612421e+00  9.63139534e-02]
 [ 5.13703041e-02  2.17957377e+00  4.59879749e-02]
 [ 1.63842633e-01  2.24289656e+00  1.13436542e-01]
 [ 2.34765574e-01  2.18316913e+00  2.16824263e-01]
 [ 3.10046792e-01  2.06629801e+00  4.53818679e-01]
 [ 1.77154735e-01  1.96469581e+00  4.59470361e-01]
 [ 3.15059096e-01  1.91914535e+00  3.43077809e-01]
 [ 2.57746018e-02  1.91194129e+00  2.49907881e-01]
 [-7.61840716e-02  2.00290108e+00  5.27887382e-02]
 [-4.38261661e-04  2.22640967e+00 -3.71758901e-02]
 [ 1.91599131e-01  2.34114265e+00  7.64061809e-02]
 [ 3.23971987e-01  2.22691369e+00  2.60671645e-01]] nm
Alch energy -4394.6582204890565
work_ncm

In [14]:
traj_nc = mdtraj.load_netcdf('test.nc', top='test.pdb')
traj_nc.image_molecules()

<mdtraj.Trajectory with 24 frames, 975 atoms, 321 residues, and unitcells at 0x7f5266684da0>

In [15]:
view = nglview.show_mdtraj(traj_nc, gui=True)
#view.camera = 'orthographic'
view.background = 'white'
# clear representations
#view.clear_representations()
# specify residue
view.add_licorice('ligand')
view.add_line('water')

view

NGLWidget(count=24)